## Issue Resolution
This is the reference implementation for [Issue Resolution Duration](https://github.com/chaoss/wg-evolution/blob/master/metrics/Issue_Resolution_Duration.md), 
a metric specified by the
[Evolution Working Group](https://github.com/chaoss/wg-evolution) of the
[CHAOSS project](https://chaoss.community).

Have a look at [README.md](../README.md) to find out how to run this notebook (and others in this directory) as well as to get a better understanding of the purpose of the implementations.

The implementation is described in two parts (see below):

* Class for computing Issue Resolution Duration
* An explanatory analysis of the class' functionality

Some more auxiliary information in this notebook:

* Examples of the use of the implementation

As discussed in the [README](../README.md) file, the scripts required to analyze the data fetched by Perceval are located in the `scripts` package. Due to python's import system, to import modules from a package which is not in the current directory, we have to either add the package to `PYTHONPATH` or simply append a `../..` to `sys.path`, so that `scripts` can be successfully imported. 

In [9]:
from datetime import datetime

import sys
sys.path.append('../..')

from implementations.scripts.issue_github import IssueGithub
from implementations.scripts.utils import read_json_file, str_to_date

In [10]:
class IssueResolutionDurationGithub(IssueGithub):
    """
    Issue Resolution Duration Metric
    """

    def _flatten(self, item):
        flat = super()._flatten(item)

        if flat:
            flat = flat[0]
        else:
            return flat

        if flat['current_status'] != 'closed':
            return []

        closed_date = str_to_date(item['data']['closed_at'])
        flat.update(
            closed_date=closed_date,
            resolution_duration=(closed_date - flat['created_date']).days)

        return [flat]

    def compute(self):
        resolution_durations = [item['resolution_duration'] for item in self.items]
        return sum(resolution_durations) / len(resolution_durations)

    def compute_max(self):
        resolution_durations = [item['resolution_duration'] for item in self.items]
        return max(resolution_durations)

    def compute_min(self):
        resolution_durations = [item['resolution_duration'] for item in self.items]
        return min(resolution_durations)

    def __str__(self):
        return "Issue Resolution Duration Metric for Github"

## Performing the Analysis
We'll perform two kinds of analysis here:
- Counting the average age of open issues
- Change of average open issue age over time

### Computing the average resolution duration of issues
First, we read the JSON file `issues.json`, present in the `implementations` directory, one level up. We make use of the `read_json_file` utility function.

In [11]:
items = read_json_file('../issues.json')

Let's use the `compute` method to compute to calculate the average of resolution duration of issues. First, we will do it without passing any since and until dates. 
Next, we can pass in the start and end dates as a tuple. The format would be `%Y-%m-%d`.

Lets calculate the average age for all open issues first. Then, we can do it by passing a start date. Here, only those issues will be considered that were created after the start date we passed via the variable `date_since`. 

While printing the output, we will keep the precision to only two decimals. 

In [12]:
date_since = datetime.strptime("2018-09-07", "%Y-%m-%d")
issue_resolution_duration = IssueResolutionDurationGithub(items)
print('The average resolution duration of all issues is {:.2f} days.'.format(issue_resolution_duration.compute()))

issue_resolution_duration = IssueResolutionDurationGithub(items, (date_since, None))
print('The average resolution duration of issues created after 2018-09-07 is {:.2f} days.'.format(issue_resolution_duration.compute()))

The average resolution duration of all issues is 22.47 days.
The average resolution duration of issues created after 2018-09-07 is 9.73 days.
